### Carregando os dados

Vimos no post anterior como carregar dados de um arquivo csv. Vamos ver quais tipos de dados temos disponívies e como estão organizados.

In [1]:
import pandas as pd
import numpy as np
election_data = pd.read_csv("primary_results.csv", delimiter=',')
election_data.head()

,state,state_abbreviation,county,fips,party,candidate,votes,fraction_votes
0,Alabama,AL,Autauga,1001.0,Democrat,Bernie Sanders,544,0.182
1,Alabama,AL,Autauga,1001.0,Democrat,Hillary Clinton,2387,0.800
2,Alabama,AL,Baldwin,1003.0,Democrat,Bernie Sanders,2694,0.329
3,Alabama,AL,Baldwin,1003.0,Democrat,Hillary Clinton,5290,0.647
4,Alabama,AL,Barbour,1005.0,Democrat,Bernie Sanders,222,0.078


Temos o número de votos de cada candidato por condado de todos os estados dos Estado Unidos. A primeira informação que podemos extrair é qual foi o total de votos nas eleições primárias.

In [2]:
election_data['votes'].sum()

56759187

Se quisermos saber os nomes de todos os candidatos que concorreram a essas eleições, podemos chamar a função [*unique()*](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.unique.html) na coluna *candidate*. Essa função filtra os dados extraindo os valores únicos de uma linha ou coluna. Os resultados são apresentados na ordem em que aparecem:

In [3]:
for candidate in election_data.candidate.unique():
    print(candidate)

Bernie Sanders
Hillary Clinton
Ben Carson
Donald Trump
John Kasich
Marco Rubio
Ted Cruz
 Uncommitted
Martin O'Malley
Carly Fiorina
Chris Christie
Jeb Bush
Mike Huckabee
Rand Paul
Rick Santorum
 No Preference


### Tabela Pivô

A próxima informação que eu quero extrair é o **total de votos que cada candidato recebeu em cada um dos estados do país**. Para conseguir isso, vamos usar uma "tabela pivô" ([*pivot table*](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html)), que é chamada de Tabela Dinâmica na tradução do Microsoft Excel. Não quero aqui trazer o conceito detalhado do que é uma tabela pivô, mas é basicamente uma tabela que sumariza dados de outra tabela. Você pode ver mais sobre isso na [wikipedia](https://en.wikipedia.org/wiki/Pivot_table) e no post que me baseei pra entender mais sobre o assunto: [Pandas Pivot Table Explained](http://pbpython.com/pandas-pivot-table-explained.html).

Vamos ver o resultado da execução da função e depois comentamos sobre o funcionamento dela. Acho que assim será mais fácil da gente entender.

In [4]:
votos_candidato = pd.pivot_table(election_data,index=["state","party","candidate"],
               values=["votes"],
               aggfunc={"votes": np.sum},fill_value=0)
votos_candidato

votes
state         party      candidate               
Alabama       Democrat   Bernie Sanders     76399
                         Hillary Clinton   309928
              Republican Ben Carson         87517
                         Donald Trump      371735
                         John Kasich        37970
                         Marco Rubio       159802
                         Ted Cruz          180608
Alaska        Democrat   Bernie Sanders       440
                         Hillary Clinton       99
              Republican Ben Carson          2401
                         Donald Trump        7346
                         John Kasich          892
                         Marco Rubio         3318
                         Ted Cruz            7973
Arizona       Democrat   Bernie Sanders    163400
                         Hillary Clinton   235697
              Republican Donald Trump      249916
                         John Kasich        53040
                         Ted Cruz          132147
Arkansas      Democrat   Bernie Sanders     64868
                         Hillary Clinton   144580
              Republican Ben Carson         23173
                         Donald Trump      133144
                         John Kasich        15098
                         Marco Rubio       101235
                         Ted Cruz          123873
California    Democrat   Bernie Sanders   1502043
                         Hillary Clinton  1940580
              Republican Donald Trump     1174829
                         John Kasich       176620
...                                           ...
Vermont       Republican Marco Rubio        11778
                         Ted Cruz            5929
Virginia      Democrat   Bernie Sanders    275507
                         Hillary Clinton   503358
              Republican Ben Carson         60093
                         Donald Trump      355960
                         John Kasich        96519
                         Marco Rubio       327042
                         Ted Cruz          173193
Washington    Democrat   Bernie Sanders     19159
                         Hillary Clinton     7140
              Republican Donald Trump      403003
                         John Kasich        52129
                         Ted Cruz           55719
West Virginia Democrat   Bernie Sanders    123860
                         Hillary Clinton    86354
              Republican Donald Trump      156245
                         John Kasich        13685
                         Ted Cruz           18208
Wisconsin     Democrat   Bernie Sanders    567936
                         Hillary Clinton   432767
              Republican Donald Trump      386370
                         John Kasich       155200
                         Ted Cruz          531129
Wyoming       Democrat   Bernie Sanders       156
                         Hillary Clinton      124
              Republican Donald Trump          70
                         John Kasich            0
                         Marco Rubio          189
                         Ted Cruz             644

[290 rows x 1 columns]

O que estamos fazendo aqui é criando uma nova tabela, cujos índices são três colunas: "*state*", "*party*" e "*candidate*". Nós informamos isso através parâmetro "*index*" da função. Os valores que desejamos agregar estão na coluna "*votes*". Além disso, o que queremos é somar a quantidade de votos, então passamos a função de soma ao parâmetro "*aggfunc*". Se não informarmos qual função o pandas deve usar, ele vai agregar calculando a média entre os valores ("*np.mean*").

O resultado é que temos todos os votos de um candidato contabilizados por estado e podemos facilmente visualizar quais candidatos foram mais votados ou menos votados em cada estado.

### Agrupamentos (groupby)

In [5]:
election_data['Rank'] = election_data.groupby(['county', 'party'])['votes'].rank(ascending=False)
election_data[election_data['county']=='Autauga']

,state,state_abbreviation,county,fips,party,candidate,votes,fraction_votes,Rank
0,Alabama,AL,Autauga,1001.0,Democrat,Bernie Sanders,544,0.182,2.0
1,Alabama,AL,Autauga,1001.0,Democrat,Hillary Clinton,2387,0.800,1.0
134,Alabama,AL,Autauga,1001.0,Republican,Ben Carson,1764,0.146,4.0
135,Alabama,AL,Autauga,1001.0,Republican,Donald Trump,5387,0.445,1.0
136,Alabama,AL,Autauga,1001.0,Republican,John Kasich,421,0.035,5.0
137,Alabama,AL,Autauga,1001.0,Republican,Marco Rubio,1785,0.148,3.0
138,Alabama,AL,Autauga,1001.0,Republican,Ted Cruz,2482,0.205,2.0


In [6]:
new_data = election_data.groupby(['state', 'party', 'candidate']).sum()
new_data = new_data.reset_index()
new_data['Rank'] = new_data.groupby(['state', 'party'])['votes'].rank(ascending=False)
pivot_data = pd.pivot_table(new_data,index=["state","party","candidate"],
               values=["votes", "Rank"],
               fill_value=0)
pivot_data

Rank    votes
state         party      candidate                     
Alabama       Democrat   Bernie Sanders      2    76399
                         Hillary Clinton     1   309928
              Republican Ben Carson          4    87517
                         Donald Trump        1   371735
                         John Kasich         5    37970
                         Marco Rubio         3   159802
                         Ted Cruz            2   180608
Alaska        Democrat   Bernie Sanders      1      440
                         Hillary Clinton     2       99
              Republican Ben Carson          4     2401
                         Donald Trump        2     7346
                         John Kasich         5      892
                         Marco Rubio         3     3318
                         Ted Cruz            1     7973
Arizona       Democrat   Bernie Sanders      2   163400
                         Hillary Clinton     1   235697
              Republican Donald Trump        1   249916
                         John Kasich         3    53040
                         Ted Cruz            2   132147
Arkansas      Democrat   Bernie Sanders      2    64868
                         Hillary Clinton     1   144580
              Republican Ben Carson          4    23173
                         Donald Trump        1   133144
                         John Kasich         5    15098
                         Marco Rubio         3   101235
                         Ted Cruz            2   123873
California    Democrat   Bernie Sanders      2  1502043
                         Hillary Clinton     1  1940580
              Republican Donald Trump        1  1174829
                         John Kasich         2   176620
...                                        ...      ...
Vermont       Republican Marco Rubio         3    11778
                         Ted Cruz            4     5929
Virginia      Democrat   Bernie Sanders      2   275507
                         Hillary Clinton     1   503358
              Republican Ben Carson          5    60093
                         Donald Trump        1   355960
                         John Kasich         4    96519
                         Marco Rubio         2   327042
                         Ted Cruz            3   173193
Washington    Democrat   Bernie Sanders      1    19159
                         Hillary Clinton     2     7140
              Republican Donald Trump        1   403003
                         John Kasich         3    52129
                         Ted Cruz            2    55719
West Virginia Democrat   Bernie Sanders      1   123860
                         Hillary Clinton     2    86354
              Republican Donald Trump        1   156245
                         John Kasich         3    13685
                         Ted Cruz            2    18208
Wisconsin     Democrat   Bernie Sanders      1   567936
                         Hillary Clinton     2   432767
              Republican Donald Trump        2   386370
                         John Kasich         3   155200
                         Ted Cruz            1   531129
Wyoming       Democrat   Bernie Sanders      1      156
                         Hillary Clinton     2      124
              Republican Donald Trump        3       70
                         John Kasich         4        0
                         Marco Rubio         2      189
                         Ted Cruz            1      644

[290 rows x 2 columns]

In [7]:
new_data[new_data['Rank'] == 1]['candidate'].value_counts()

Donald Trump       36
Hillary Clinton    28
Bernie Sanders     21
Ted Cruz            9
John Kasich         1
Name: candidate, dtype: int64